In [1]:
import nltk
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import pickle
import numpy as np
import pandas as pd
import json
import random

In [2]:
from tensorflow.keras.models import load_model

In [3]:
model = load_model('chatbot_model.keras')

In [4]:
with open('intent.json','r') as f:
    data = json.load(f)
df = pd.DataFrame(data['intents'])
df

,tag,patterns,responses
0,greeting,"[Hi, Hello, Hey there, Good morning, How are y...","[Hello! How can I help you today?, Hi there! W..."
1,weather,"[What's the weather like today?, Is it going t...",[The weather today is sunny with a high of 25°...
2,restaurant_recommendations,"[Can you recommend a good restaurant?, I'm loo...",[Sure! How about [Restaurant A]? They have gre...
3,product_information,"[Tell me about your products., What types of p...",[We offer a wide range of products including e...
4,booking_flights,"[I want to book a flight., Can you help me fin...",[Sure! Where would you like to fly to and from...
5,tech_support,"[I'm having trouble with my [device]., How do ...",[I'm sorry to hear that. What seems to be the ...
6,order_tracking,"[Where is my order?, Can you track my package?...",[Please provide your order number so I can che...
7,account_support,"[I forgot my password., How do I change my ema...",[You can reset your password through the passw...
8,appointment_booking,"[How do I schedule an appointment?, Can I book...",[You can schedule an appointment online throug...
9,general_info,"[What can you tell me about your company?, Whe...","[We are a company specializing in [industry].,..."


In [5]:

with open('intent.json', encoding = 'utf8') as json_file:
    intents = json.load(json_file)

# Load the words and classes pickle files
with open('word.pkl', 'rb') as words_file:
    words = pickle.load(words_file)

with open('class.pkl', 'rb') as classes_file:
    classes = pickle.load(classes_file)

In [6]:
doc = []
for index, row in df.iterrows():
    patterns = row['patterns']
    tag = row['tag']

    # Iterate over questions in each pattern
    for question in patterns:
        
        # Append [question, tag] to documents
        doc.append([question, tag])
doc = pd.DataFrame(doc)
X = list(doc[0])
y = list(doc[1])

data = {
    'question': X,
    'tag':y
}

df = pd.DataFrame(data)
# question_vectors = vectorizer.fit_transform(df['question']).toarray()
intents

{'intents': [{'tag': 'greeting',
   'patterns': ['Hi', 'Hello', 'Hey there', 'Good morning', 'How are you?'],
   'responses': ['Hello! How can I help you today?',
    'Hi there! What can I do for you?',
    "Hey! How's it going?",
    'Good morning! How can I assist you?']},
  {'tag': 'weather',
   'patterns': ["What's the weather like today?",
    'Is it going to rain tomorrow?',
    "How's the weather in [city]?",
    "Tell me about today's weather forecast.",
    'Will it be sunny this weekend?'],
   'responses': ['The weather today is sunny with a high of 25°C.',
    "Tomorrow's forecast predicts rain in the afternoon.",
    "In [city], it's currently 20°C and partly cloudy.",
    'Today will be sunny with a slight chance of showers.',
    'This weekend looks like it will be sunny and warm.']},
  {'tag': 'restaurant_recommendations',
   'patterns': ['Can you recommend a good restaurant?',
    "I'm looking for Italian food.",
    'Any sushi restaurants nearby?',
    'Where can I fin

In [7]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(vocabulary = words)
tfidf_vectors = vectorizer.fit_transform((df['question']))
import numpy as np

In [8]:
# doc = []
# for index, row in df.iterrows():
#     patterns = row['patterns']
#     tag = row['tag']

#     # Iterate over questions in each pattern
#     for question in patterns:
        
#         # Append [question, tag] to documents
#         doc.append([question, tag])
# doc = pd.DataFrame(doc)
# X = list(doc[0])
# y = list(doc[1])

# data = {
#     'question': X,
#     'tag':y
# }

# df = pd.DataFrame(data)
# question_vectors = vectorizer.fit_transform(df['question']).toarray()

In [9]:
# # to correlate to the best sentence
# from sklearn.metrics.pairwise import cosine_similarity

# # Example set of sentences
# sentences = [
#     "The cat sat on the mat.",
#     "The dog played in the garden.",
#     "Birds chirped in the trees.",
#     "The sun is shining brightly.",
#     "Children are playing in the park."
# ]

# # Reference sentence
# reference_sentence = "The cat sat on the mat."

# # Function to find the sentence with the highest cosine similarity to the reference sentence
# def find_best_match(reference_sent, sentences):
#     # Initialize TF-IDF vectorizer
#     vectorizer = TfidfVectorizer()

#     # Fit and transform sentences to TF-IDF vectors
#     tfidf_matrix = vectorizer.fit_transform([reference_sent] + sentences)

#     # Calculate cosine similarity between the reference sentence and all sentences
#     cosine_sims = cosine_similarity(tfidf_matrix[0], tfidf_matrix[1:]).flatten()

#     # Find the index of the sentence with the highest similarity (excluding the reference sentence itself)
#     best_match_index = cosine_sims.argmax() + 1  # +1 to adjust for the reference sentence at index 0

#     # Return the best matching sentence and its similarity score
#     best_match_sentence = sentences[best_match_index]
#     similarity_score = cosine_sims[best_match_index]

#     return best_match_sentence, similarity_score


In [10]:
# from sklearn.feature_extraction.text import TfidfVectorizer
# vectorizer = TfidfVectorizer(vocabulary = words)
# tfidf_vectors = vectorizer.fit_transform((df['question']))
# import numpy as np

In [11]:

def get_response(ints, intents_json):
    tag = ints[0]['intent']
    list_of_intents = intents_json['intents']
    for i in list_of_intents:
        if i['tag'] == tag:

            
            result = i['responses'][random.randint(1, 4)]
            break
    return result

    

In [12]:
def get_responses(tag, data):
    intents = data['intents']
    for intent in intents:
        if intent['tag'] == tag:
            return intent['responses']
    return []


In [ ]:
while True:
    print('User:')
    user_input = input()
    
    # Transform user input to match the trained model's input format
    question_vector = vectorizer.transform([user_input]).toarray()
    
    # Predict the class
    tag = model.predict(question_vector).argmax(axis=1)[0]
    
    # Find the corresponding intent tag
    predicted_intent = classes[tag]
    print(predicted_intent)
   
    # Get the response from the intent
    response = get_response([{'intent': predicted_intent, 'probability': '1.0'}], intents)
    
    print("Chatbot:", response)

User:


 how are you?


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 318ms/step
job_application
Chatbot: Our recruitment process involves [steps/process details].
User:


 i am jobless


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
event_information
Chatbot: We regularly host workshops and seminars on [topics]. Stay tuned for updates.
User:


...

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
job_application
Chatbot: You can apply for a job through our careers page on our website.
User:


 how do i get a job


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
job_application
Chatbot: Yes, we offer internships in [department/role].
User:


 i also want to hire a lawyer


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
booking_flights
Chatbot: Let me check for flights to [destination].
User:
